In [20]:
import SimpleITK as sitk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
from scipy import signal
import cv2
import random

from random import randrange
from numpy import linalg 
from scipy import signal
from pylab import *
from PIL import Image

## full matrix
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
## imshow problem
import tkinter
import matplotlib
matplotlib.use('TkAgg')

In [2]:
dir1='/home/mahdi/Desktop/Im6.png'
dir2='/home/mahdi/Desktop/Im7.png'

In [32]:
directory='/home/mahdi/python codes/final version/SP_S05_D1_RND.nii'
I = sitk.ReadImage(directory)
I = sitk.GetArrayFromImage(I)
I=I.astype(np.float32)

def mask(I,volume, layer):
        
        if volume<10:
            name=str(0)+str(0)+str(volume)
        if 9<volume<100:
            name=str(0)+str(volume)
        if 99<volume<1000:
            name=str(volume)
            
        g=I[volume,layer,:,:]
        df = pd.read_csv('/home/mahdi/python codes/centerline_case2/centerline_volume'+name+'.csv', header=None)
        df.columns=['x','y','delete']
        df=df[['x','y']]
        c=df.loc[layer]
        x=int(c['x'])
        y=int(c['y'])
        f=g[y-15:y+15,x-15:x+15]
        return f

In [22]:
I.shape

(318, 10, 110, 110)

In [23]:
def LucasKanadeee(I1, I2,d,q):
    I1 = np.array(I1)
    I2 = np.array(I2)
    S  = np.shape(I1)
    
    smooth_kernel=5
    I1_smooth = cv2.GaussianBlur(I1,(smooth_kernel,smooth_kernel), 0)
    I2_smooth = cv2.GaussianBlur(I2 ,(smooth_kernel,smooth_kernel), 0)
    
    Ix = signal.convolve2d(I1_smooth,[[-0.25,0.25] , [-0.25,0.25]],'same') + signal.convolve2d(I2_smooth,[[-0.25,0.25],[-0.25,0.25]] ,'same')
    Iy = signal.convolve2d(I1_smooth,[[-0.25,-0.25], [0.25,0.25]],'same') + signal.convolve2d(I2_smooth ,[[-0.25,-0.25],[0.25,0.25]]  ,'same')
    It = signal.convolve2d(I1_smooth,[[0.25,0.25]  , [0.25,0.25]],'same') + signal.convolve2d(I2_smooth ,[[-0.25,-0.25],[-0.25,-0.25]],'same')
    
    
    features = cv2.goodFeaturesToTrack(I1_smooth, 1000, q, d)
    features = np.int0(features)
      

    plt.subplot(1,3,1)
    plt.title('Frame 1')
    plt.imshow(I1, cmap = cm.gray)
    plt.subplot(1,3,2)
    plt.title('Frame 2')
    plt.imshow(I2, cmap = cm.gray)#plotting the features in frame1 and plotting over the same
    
 
    u = np.nan*np.ones(S)
    v = np.nan*np.ones(S)
    
    for l in features:
        j,i = l.ravel()
        IX = ([Ix[i-1,j-1],Ix[i,j-1],Ix[i+1,j+1],Ix[i-1,j],Ix[i,j],Ix[i+1,j],Ix[i-1,j+1],Ix[i,j+1],Ix[i+1,j-1]])
        IY = ([Iy[i-1,j-1],Iy[i,j-1],Iy[i+1,j+1],Iy[i-1,j],Iy[i,j],Iy[i+1,j],Iy[i-1,j+1],Iy[i,j+1],Iy[i+1,j-1]])
        IT = ([It[i-1,j-1],It[i,j-1],It[i+1,j+1],It[i-1,j],It[i,j],It[i+1,j],It[i-1,j+1],It[i,j+1],It[i+1,j-1]])
        #IX = ([Ix[i-1,j-1],Ix[i-1,j],Ix[i-1,j+1],Ix[i,j-1],Ix[i,j],Ix[i,j+1],Ix[i+1,j-1],Ix[i+1,j],Ix[i+1,j+1]])
        #IY = ([Ix[i-1,j-1],Ix[i-1,j],Ix[i-1,j+1],Ix[i,j-1],Ix[i,j],Ix[i,j+1],Ix[i+1,j-1],Ix[i+1,j],Ix[i+1,j+1]])
        #IT = ([Ix[i-1,j-1],Ix[i-1,j],Ix[i-1,j+1],Ix[i,j-1],Ix[i,j],Ix[i,j+1],Ix[i+1,j-1],Ix[i+1,j],Ix[i+1,j+1]])
        # Using the minimum least squares solution approach
        LK = (IX,IY)
        LK = matrix(LK)
        LK = array(LK)
        
        #Psedudo Inverse
        A2 = np.linalg.pinv(LK)
        IT=np.array(IT)
        b=-IT
        
        (u[i,j],v[i,j]) = np.dot(A2.T,b) # we have the vectors with minimized square error
        
    a=0     
    colors = "bgrcmyw"
    color_index = random.randrange(0,7)
    c=colors[color_index]
    #======= Plotting the vectors on the image========
    plt.subplot(1,3,3)
    plt.title('Vector plot of Optical Flow of good features')
    plt.imshow(I1,cmap = cm.gray)
    for i in range(S[0]):
        for j in range(S[1]):
            if (abs(u[i,j])>t or abs(v[i,j])>t): # setting the threshold to plot the vectors
                u[i,j]=nan
                v[i,j]=nan
                a=a+1
            plt.arrow(j,i,u[i,j],v[i,j],head_width = 1, head_length = 2, color = c)
                
    plt.show()
    print('number of failed in threshold=',a)
    return

# make fake image

In [24]:
i=np.zeros((500,500))
np.array(i)
i[50,51]=1
i[55,55]=1
i=i.astype(np.float32)


j=np.zeros((500,500))
np.array(j)
j[56,50]=1
j[54,53]=1
j=j.astype(np.float32)

###################################### display ##################################
#plt.imshow(i,cmap='gray')
#plt.figure()
#plt.imshow(j,cmap='gray')
#plt.show()

In [19]:
import random
from random import randrange
#%matplotlib inline
t=4
############## run for fake image#################
LucasKanadeee(i,j,1,.0001)

################ run for real image ################ frame=159 , slice=5 ###### and ##### frame=100 , slice=5
#LucasKanadeee(I[158,5,:,:], I[159,5,:,:],1,.0001)

################## with mask ############################
#LucasKanadeee(mask(I,158,5),mask(I,159,5),1,.0001)



number of failed in threshold= 0


In [8]:
Im1=plt.imread(dir1)
Im2=plt.imread(dir2)

LucasKanadeee(Im1,Im2,1,0.0001)

number of failed in threshold= 10


# feature test 

In [9]:
d=1
q1=mask(I,158,5)
q2=mask(I,159,5)
I1 = np.array(q1)
I2 = np.array(q2)

smooth_kernel=3
I1_smooth = cv2.GaussianBlur(I1,(smooth_kernel,smooth_kernel), 0)
I2_smooth = cv2.GaussianBlur(I2 ,(smooth_kernel,smooth_kernel), 0)
   
features1 = cv2.goodFeaturesToTrack(I1_smooth, 1000, .0001, d)
features1 = np.int0(features1)

   
features2 = cv2.goodFeaturesToTrack(I2_smooth, 1000, 0.0001, d)
features2 = np.int0(features2)


features1.shape

(13, 1, 2)

In [58]:
f1=I[0,5,:,:]
f2=I[1,5,:,:]
f11=mask(I,0,5)
f22=mask(I,1,5)


In [61]:
plt.imshow(f11-f22,cmap='gray')
plt.figure()
plt.imshow(f1-f2,cmap='gray')
plt.figure()
plt.imshow(abs(f1-f2),cmap='gray')
plt.figure()
plt.imshow(I[0,5,:,:],cmap='gray')
plt.show()

In [55]:
f11-f22

array([[   4.,  -44.,  -35.,   52.,   48.,  -52.,    1.,  -23.,  -12.,
          17.,   62.,    8.,   -5.,   16.,    6.,   13.,    2.,   34.,
         -12.,  -13.,    2.,  -54.,  -10.,  -34.,   18.,   55.,   15.,
         -26.,  -25.,   57.],
       [  -5.,    6.,   -3.,    9.,    7.,  -48.,  -42.,   11.,  -18.,
         -55.,   54.,    7.,  -43.,   -9.,   -7.,  -26.,    4.,   14.,
          11.,  -11.,   -1.,  -35.,   -1.,  -85.,   -3.,  -47.,   25.,
          10.,   13.,  -32.],
       [   0.,    8.,   23.,   -3.,   26.,   22.,  -54.,    7.,   20.,
          -2.,   42.,   35.,   23.,    8.,   -1.,  -19.,   21.,    6.,
           2.,    3.,    2.,   34.,   26.,  -24.,   15.,  -16.,  -13.,
         -19.,    5.,   26.],
       [  45.,   11.,   18.,   35.,  -17.,    1.,  -47.,   -9.,   37.,
          59.,  -23.,    4.,   42.,   -2.,   -3.,  -15.,    1.,   -1.,
         -12.,    7.,  -27.,    7.,  -30.,  -19.,  -24.,    2.,    2.,
          35.,  -18.,    4.],
       [ -55.,   19.,  -45.,

# add sense